# Example 2: Attach SAE and Save Top Texts for Neurons

This notebook demonstrates how to:
1. Load a trained SAE model from the previous example
2. Attach the SAE to a language model
3. Enable text tracking to collect top activating texts for each neuron
4. Run inference on new text data to collect neuron-text associations
5. Save the collected top texts for use in the next example

The top texts will be saved to `outputs/top_texts.json` for use in the next example.


In [1]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
import json
from pathlib import Path
from datetime import datetime

from amber.store import LocalStore
from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from amber.mechanistic.autoencoder.autoencoder import Autoencoder

print("✅ Imports completed")


/Users/adam/Projects/Inzynierka/codebase/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports completed


In [2]:
# Load metadata from previous example
metadata_path = Path("outputs/training_metadata.json")
if not metadata_path.exists():
    print("❌ Error: training_metadata.json not found!")
    print("   Please run 01_train_sae_model.ipynb first")
    raise FileNotFoundError("training_metadata.json not found")

with open(metadata_path, "r") as f:
    metadata = json.load(f)

print("📋 Loaded training metadata:")
for key, value in metadata.items():
    if key != "training_history":  # Skip large training history
        print(f"   {key}: {value}")
print()


📋 Loaded training metadata:
   run_id: sae_training_20251101_011640
   layer_signature: gpt2lmheadmodel_transformer_h_0_attn_c_attn
   hidden_dim: 6
   n_latents: 24
   model_id: sshleifer/tiny-gpt2
   dataset: roneneldan/TinyStories
   data_limit: 1000
   sae_model_path: outputs/sae_model.pt
   store_dir: outputs/store
   cache_dir: outputs/cache



In [3]:
# Configuration
MODEL_ID = metadata["model_id"]
LAYER_SIGNATURE = metadata["layer_signature"]
SAE_MODEL_PATH = Path(metadata["sae_model_path"])
CACHE_DIR = Path(metadata["cache_dir"])
STORE_DIR = Path(metadata["store_dir"])

# New dataset for text collection (different from training data)
HF_DATASET = "roneneldan/TinyStories"
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 500  # Smaller dataset for text collection
MAX_LENGTH = 64

# Text tracking configuration
TOP_K = 10  # Number of top texts to track per neuron
NEGATIVE_TRACKING = False  # Track positive activations

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("🔗 Starting SAE Attachment and Text Collection Example")
print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📊 Dataset: {HF_DATASET}")
print(f"🎯 Target layer: {LAYER_SIGNATURE}")
print(f"🧠 SAE model: {SAE_MODEL_PATH}")
print()


🔗 Starting SAE Attachment and Text Collection Example
📱 Using device: cpu
🔧 Model: sshleifer/tiny-gpt2
📊 Dataset: roneneldan/TinyStories
🎯 Target layer: gpt2lmheadmodel_transformer_h_0_attn_c_attn
🧠 SAE model: outputs/sae_model.pt



In [4]:
# Step 1: Load language model
print("📥 Loading language model...")

# Load model and move to device
model = LanguageModel.from_huggingface(MODEL_ID)
model.model.to(DEVICE)

# Optional: set experiment metadata
model.context.experiment_name = "sae_attachment"
model.context.run_id = f"attachment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
model.context.max_length = MAX_LENGTH

print(f"✅ Model loaded: {model.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"🔧 Context: {model.context.experiment_name}/{model.context.run_id}")


📥 Loading language model...
✅ Model loaded: sshleifer_tiny-gpt2
📱 Device: cpu
🔧 Context: sae_attachment/attachment_20251101_011727


In [5]:
# Step 2: Load trained SAE
print("📥 Loading trained SAE...")
if not SAE_MODEL_PATH.exists():
    print(f"❌ Error: SAE model not found at {SAE_MODEL_PATH}")
    print("   Please run 01_train_sae_model.ipynb first")
    raise FileNotFoundError(f"SAE model not found at {SAE_MODEL_PATH}")

sae, dataset_normalize, dataset_target_norm, dataset_mean = Autoencoder.load_model(SAE_MODEL_PATH)
sae.to(DEVICE)

# Update SAE context with current experiment info
sae.context.experiment_name = "sae_attachment"
sae.context.run_id = f"attachment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

print(f"✅ SAE loaded: {metadata['hidden_dim']} → {metadata['n_latents']} → {metadata['hidden_dim']}")
print(f"🔧 Context: {sae.context.experiment_name}/{sae.context.run_id}")
print(f"📊 Dataset normalization: {dataset_normalize}")
if dataset_target_norm is not None:
    print(f"📊 Target norm: {dataset_target_norm}")
if dataset_mean is not None:
    print(f"📊 Dataset mean: {dataset_mean.shape}")


2025-11-01 01:17:27,719 [INFO] amber.mechanistic.autoencoder.autoencoder: 
Loaded model from outputs/sae_model.pt
n_latents=24, n_inputs=6, activation=TopK_8, tied=False


📥 Loading trained SAE...
✅ SAE loaded: 6 → 24 → 6
🔧 Context: sae_attachment/attachment_20251101_011727
📊 Dataset normalization: False
📊 Target norm: False
📊 Dataset mean: torch.Size([6])


In [6]:
# Step 3: Use SAE's built-in concepts for text tracking
print("🔗 Using SAE's built-in concepts for text tracking...")

# Set the language model and layer signature on the SAE's context
sae.context.lm = model
sae.context.lm_layer_signature = LAYER_SIGNATURE

# Enable text tracking via the SAE helper (sets context flags)
sae.enable_text_tracking(
    k=TOP_K,
    negative=NEGATIVE_TRACKING,
)

print(f"✅ Text tracking enabled: top-{TOP_K} {'negative' if NEGATIVE_TRACKING else 'positive'} activations")
print(f"🔧 Context: {sae.context.experiment_name}/{sae.context.run_id}")


🔗 Using SAE's built-in concepts for text tracking...
✅ Text tracking enabled: top-10 positive activations
🔧 Context: sae_attachment/attachment_20251101_011727


In [7]:
# Step 4: Load dataset for text collection
print("📥 Loading dataset for text collection...")
dataset = TextSnippetDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)
print(f"✅ Loaded {len(dataset)} text samples")


📥 Loading dataset for text collection...


Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 224006.84 examples/s]

✅ Loaded 500 text samples


In [8]:
# Step 5: Run inference to collect top texts
print("🔍 Running inference to collect top texts...")

batch_size = 16
total_batches = (len(dataset) + batch_size - 1) // batch_size

for i in range(0, len(dataset), batch_size):
    batch_end = min(i + batch_size, len(dataset))
    batch_texts = [dataset[j] for j in range(i, batch_end)]

    # Run forward pass to trigger text tracking
    model.forwards(batch_texts)

    if (i // batch_size + 1) % 10 == 0:
        print(f"   Processed {batch_end}/{len(dataset)} samples...")

print("✅ Text collection completed!")


🔍 Running inference to collect top texts...
   Processed 160/500 samples...
   Processed 320/500 samples...
   Processed 480/500 samples...
✅ Text collection completed!


In [9]:
# Step 6: Analyze collected top texts
print("📊 Analyzing collected top texts...")

# Get top texts for a few neurons as examples
example_neurons = [0, 1, 2, 5, 10]  # Show first few neurons
total_neurons_with_texts = 0

for neuron_idx in example_neurons:
    if neuron_idx < metadata["n_latents"]:
        top_texts = sae.concepts.get_top_texts_for_neuron(neuron_idx)
        if top_texts:
            total_neurons_with_texts += 1
            print(f"🧠 Neuron {neuron_idx}: {len(top_texts)} texts")
            for j, nt in enumerate(top_texts[:3]):  # Show top 3
                print(f"   {j + 1}. '{nt.text}' (score: {nt.score:.4f})")
            if len(top_texts) > 3:
                print(f"   ... and {len(top_texts) - 3} more")
            print()

# Count total neurons with texts
for neuron_idx in range(metadata["n_latents"]):
    if sae.concepts.get_top_texts_for_neuron(neuron_idx):
        total_neurons_with_texts += 1

print(f"📈 Summary: {total_neurons_with_texts}/{metadata['n_latents']} neurons have collected texts")


📊 Analyzing collected top texts...
🧠 Neuron 0: 10 texts
   1. 'Mummy and Daddy were picking flowers in the garden. Mummy picked a red daisy, Daddy picked a purple thistle and the little girl picked a beautiful lily. The lily was her favorite because it was so fluffy and white and the aroma was heavenly. 

Daddy said, "Let's bring this lily inside and put it on the windowsill." 

Mummy said, "How about we make it a surprise?" 

So the family all went inside and the little girl put the lily on the windowsill. 

When she stepped back to admire her work, she noticed a bright yellow butterfly that had landed on the lily. The little girl smiled. 

Mummy said, "Oh my, that lily looks so warm and cozy with the butterfly on top."

The little girl nodded, delighted with her surprise. And, from that day on, the warm lily became a happy reminder of the special family day.' (score: 0.0189)
   2. 'Once there was a generous bear. He liked to help others and was always very kind. But he had one habit 

In [10]:
# Step 7: Save top texts
print("💾 Saving top texts...")
sae.concepts.export_top_texts_to_json("outputs/top_texts.json")
print(f"📊 Saved texts for {total_neurons_with_texts} neurons")


💾 Saving top texts...
📊 Saved texts for 10 neurons


In [11]:
# Step 9: Save metadata for next example
attachment_metadata = {
    "model_id": MODEL_ID,
    "layer_signature": LAYER_SIGNATURE,
    "n_latents": metadata["n_latents"],
    "top_k": TOP_K,
    "negative_tracking": NEGATIVE_TRACKING,
    "dataset": HF_DATASET,
    "data_limit": DATA_LIMIT,
    "total_neurons_with_texts": total_neurons_with_texts,
    "top_texts_path": "outputs/top_texts.json",
    "sae_model_path": str(SAE_MODEL_PATH),
}

attachment_metadata_path = Path("outputs/attachment_metadata.json")
with open(attachment_metadata_path, "w") as f:
    json.dump(attachment_metadata, f, indent=2)

print(f"📋 Attachment metadata saved to: {attachment_metadata_path}")
print()
print("🎉 SAE attachment and text collection completed successfully!")
print("📝 Next: Run 03_load_and_manipulate_concepts.ipynb to load and manipulate the concepts")


📋 Attachment metadata saved to: outputs/attachment_metadata.json

🎉 SAE attachment and text collection completed successfully!
📝 Next: Run 03_load_and_manipulate_concepts.ipynb to load and manipulate the concepts
